# 一 初识并发编程
## 1. 我想对你说
通过上一阶段的学习相信你已经清楚了并发和并行的概念，那么我们为了让服务端服务于更多的客户端就要使服务端并发，如何实现并发呢？这就是我们接下来要讲解的内容：并发编程。
并发编程首先我们要讲解的是一些理论：操作系统的发展史，不要以为是历史就不重要，操作系统的发展史直接决定了后面学习并发编程的好坏，因为进程的概念起源于操作系统。可能有一部分同学以前就知道并发编程，也大概会用一些，那么我告诉你：忘记吧！

> 并发编程的真正难度不是难在写代码上面，而是难在理解上面，因为这里面全都是抽象的东西

## 2. 进程的理解
一提到并发编程，大家可能多多少少会有了解，用多进程和多线程来解决并发编程的问题，我们首先介绍的是多进程，在学习多进程之前我们应该先清楚进程的概念。
什么是进程？
根据字面意思理解，进程就是：一个正在进程的程序，指的就是一个程序的运行过程，很明显这是一个抽象的概念，是一个状态。为了对比，我们也思考一下，程序的概念。
什么是程序？
程序就是一堆代码，或者说是一堆文件。
什么叫起了一个进程？

>你下载了一个软件叫做 QQ，这就是一个程序
你运行这个 QQ 和别人的女朋友聊天，这就叫做起了一个 QQ 进程
你正在运行这个 QQ ，这个 QQ 整个的运行过程叫做一个进程

综上所述，我们可以得出一个结论：进程是一个抽象的概念。这个概念起源于操作系统，因为你电脑上面运行的程序和你自己写的 SharedNetworkDisk 程序都是要让操作系统来执行这个工作（第一章内容回顾：写程序是为了让机器取代人力，让机器无休止地为你工作，你还不给他工资），操作系统最后还是要给你的机器上面的硬件来进行具体的执行任务，这个硬件的核心也就是计算机的CPU。你写的程序不能直接操作硬件，需要调用操作系统控制硬件的接口，也就是向操作系统发请求，由操作系统控制CPU来运行你写的程序。

>一个程序的执行过程
1 最开始程序就是一个文件，他是放在硬盘里面的
2 当我启动这个程序的时候，就是把这个程序从硬盘读入内存，目的是为了让CPU从内存直接处理这个程序
3 但是应用程序并不能直接调用硬件，而是先把指令发给操作系统
4 由操作系统调用CPU执行这个程序

所以执行程序的过程其实是由操作系统的调度来完成的，那么也就可以理解为进程是由操作系统控制的。

# 二 操作系统

## 1. 操作系统简介
通过上面的图你可以看到：一个操作系统，要执行的进程肯定不只有一个， 我们先暂且说他要执行的进程是20个，但是硬件只有一个或者说只有一套，有这么多进程要执行，肯定是不能让这些进程自己去抢的，所以必须要分配一下任务，那么这个分配任务的工作一定是由操作系统来完成的。那么接下来我们就要把操作系统的工作原理搞明白。
什么是操作系统？
操作系统是位于计算机硬件与应用软件之间的，用于协调，管理和控制计算机硬件与软件资源的一种控制程序

**操作系统的两大作用

>1 把复杂丑陋的硬件操作都封装成美丽的接口，提供给应用程序使用
     说明：
    （1）假如没有操作系统，你要写程序要先研究计算机的CPU，内存，硬盘和网卡等等一系列硬件
        的工作原理，等你研究明白了，程序也没必要写了，因为公司已经倒闭了
    （2）既然每个程序员写程序都要控制硬件，那么为了提升开发效率，最好是有人专门写好这一系
        列的调用硬件的接口，写应用软件的程序员直接调用这一系列接口就可以了
    （3）至此，操作系统诞生，其实这个思想也就是我们从第二阶段的项目就开始涉猎的程序架构设计
        的思想，不要把所有的功能全部揉到一起而要分层去实现。
    （4）应用软件其实就是在调用操作系统提供的接口，比如操作硬盘的接口就是你以前学过的文件操作，
        操作网卡的接口就是你以前学过的网络编程。这些其实都是面向对象的思想之一，叫做封装。
2 把进程对硬件的竞争变得有序化
     说明：
    （1）应用程序写好了需要运行，那么也就是在操作系统上起一个进程，操作系统上面肯定有多个进
        程，如果不加以控制，就会出现以下错乱的情况
    （2）当需要打印的时候，你的 Word 程序和 Excel 程序都执行打印操作，计算机打印了一行Word
        文件，又打印了一行Excel文件，最终你的打印结果就乱了
    （3）比如说，我现在有一块蛋糕，我不切开分成块，整个蛋糕让你们大家一块吃，那么你就有可能
        咬到别人女朋友的舌头，这就尴尬了。
    （4）共享带来的问题就是竞争，竞争带来的问题就是错乱
    
## 3. 多道技术
多道技术导入

多道技术的产生背景是：想要在单核CPU的情况下实现多个进程并发执行的效果。
对于第二代计算机而言，所有的程序都是串行执行的，有几个任务要做，那么就一定要按照先后顺序，从前往后执行，直到第一个程序结束了，才会执行第二个程序。再假如如果没有多道技术的单核CPU的情况下，你的Python代码里面写了一个time.sleep(3)，那么CPU就只能干巴巴看着等着，等你三秒，或者是你写的程序里面涉及到文件操作，那么不可避免的就会产生IO时间，CPU也是干巴巴看着，等着，其实这两个操作都不会占用CPU的资源，那么CPU在这个时候不工作就是CPU资源的浪费。比如你开了一个公司，公司就招了一个员工，你要求这个员工要给你炒菜，还要给你做饭，还要给你斟茶，假如在整个炒菜的过程中，这个人都不空闲着，那么他的利用率一定高， 假如这三个工作一直不停着，那么你就让他从前往后干这肯定没毛病，但是如果在做第一个工作的时候产生了停顿，那么这个利用率肯定就会降低了，最好是在做第一个工作的时候遇到等的过程就去做第二个任务，再次遇到等的过程就去做第三个任务，这样一遇到等待就来回地切换，就能提高它的工作效率。

>多道技术
1 空间上的复用（多道程序复用内存的空间）
     说明：
         （1）将内存分为几部分，每个部分放入一个程序，这样，同一时间内存中就有了多道程序
         （2）最后执行程序的是CPU，程序一定要要先由硬盘加载到内存CPU才能执行这个程序
         （3）我们为了实现并发的效果，一定要保证CPU在多个进程之间高速切换，所以不能从硬盘读数据
2 时间上的复用（多道程序复用CPU的时间片）
     说明：
         当一个程序在等待I/O时，另一个程序可以使用CPU，如果内存中可以同时存放足够多的作业，
         则cpu的利用率可以接近100%，类似于我们小学数学所学的统筹方法
         
时间上的复用上面说的是遇到 IO 操作就会切换，但是，多道技术并不是仅仅只遇到 IO 就会切换，也有可能3个进程都没有任何的 IO 操作，全是计算的任务，比如，我给计算机三个任务：1 计算1 +++加到1万亿，2 计算1乘乘乘到1万亿，3 计算1---减到1万亿，这三个进程都是纯计算的操作，这种情况下操作系统应该怎么处理？第一个任务计算的时间较长，我们不应该让一个进程占用CPU的时间过长，当他占用的时间过长了，即使没有IO操作也要切换，对于所有的进程来说应该是雨露均沾。遇到IO切换和占用时间过长切换，这个工作是由操作系统来控制的， 这个切换工作一定要快，如果要用代码实现的话其实也就是写一个流程控制程序，Python做不了，C语言也做不了，因为都不够快，要用汇编语言去实现，所以在操作系统里面就会有一段汇编语言写的调度程序，这个调度程序始终会监测着你的机器之上运行着的进程， 一旦某个进程运行时间过长了，操作系统会强行的把它的执行权限拿走给下一个进程，一个进程遇到IO操作了，当然操作系统也会立即把CPU执行权限拿走交给其他的进程。
很明显，遇到IO切换能够提高CPU的使用率，但是如果多个进程全部都是计算操作，其中任何一个进程都没有遇到IO，操作系统也强行地把它的CPU执行权限拿走，这样虽然能够实现并发的效果，但是并不能提升CPU的效率，相反，加上进程切换的时间，一定会比原来消耗更多的时间，所以这种情况是：降低了效率，保证了并发的效果。

## 4. 程序的执行效率
假如现在你的机器上有 QQ进程，微信进程还有你自己写的 SharedNetworkDisk 进程，这三个进程在操作系统看来都是进程，CPU只有一个核（我们先研究单个CPU，研究明白了之后，多核CPU自然就明白了），那么怎么样就是你自己写的这个程序运行效率高了？一定是CPU更多的时间在运行你写的程序，你写的程序就更多的被处理了，也就是你写的程序执行效率高了，所以你应该尽可能的把CPU和你写的程序绑到一起去，CPU的分配是由操作系统来控制的， 要想占用CPU更多的时长，最直接最简单的解决方案就是尽可能减少你写的程序中的IO操作，所以，以后在写程序的时候就是要尽可能降低程序的IO操作。

## 5. 进程运行的三种状态
一个进程运行会有三种状态，这三种状态分别是运行态，阻塞态和就绪态。
![avatar](https://cdn.nlark.com/yuque/0/2019/png/281865/1554537690773-38d68991-fa72-49b7-960a-906883714083.png#align=left&display=inline&height=301&originHeight=301&originWidth=643&size=0&status=done&width=643)
正在被CPU运行的状态就是运行态，从运行态会切换到阻塞态和就绪态。就绪态就是时刻准备着被CPU运行，进程在运行中遇到IO操作不占用CPU了就会切换到阻塞态，要想重新被CPU运行就要先进入就绪状态才有可能拿到CPU的执行权限被运行。如果是从运行态直接切换到就绪态这是遇到什么问题呢？那么一定是CPU占用时间过长或者是有一个优先级比当前进程更高的进程把CPU抢走了（这种情况后面有讲解），由运行态切换到就绪态这是不可避免的， 但是你可以做到尽可能的减少阻塞状态，这也就意味着进程最大限度的切换到了就绪态， 对于操作系统来说，就绪态是一个时刻准备好的状态，那么他就会尽可能多的去分配CPU的执行权限。所以，再重复一遍：提成程序的执行效率，就是尽可能降低IO操作。
关于串行和阻塞这里需要注意一点：串行执行也有可能会遇到一个进程等待另外一个进程的情况，但是这种等待与阻塞是没有任何关系的。